### Install Dependencies
Make sure you have the required dependencies installed. You can do this by running the following command in your terminal:

In [1]:
%pip install --only-binary=:all: tiktoken
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.9 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.1 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Install required libraries
%pip install -qU \
    langchain==0.3.* \
    langchain_openai==0.3.* \
    langchain_community \
    unstructured[md]==0.17.* \
    langgraph==0.4.* \
    websockets==15.0.*

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
import requests
import os

from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
if (openai_api_key):
    print("------------------- Loaded Key -------------------")
# print(openai_api_key)  # optional: check it loaded correctly

# Load the OpenAPI specification from the running ticketing system
root = "http://localhost:3000"
api_spec_url = f"{root}/api/docs/openapi.json"

# Download and parse the OpenAPI spec
response = requests.get(api_spec_url)
data = response.json()
data['servers'] = [{'url': root}]
openapi_spec = reduce_openapi_spec(data, dereference=False)

### Try to get it to categorize all tickets
requests_wrapper = RequestsWrapper()
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

import requests

agent = planner.create_openapi_agent(
    api_spec=openapi_spec,
    requests_wrapper=requests_wrapper,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    handle_parsing_errors=True,
    allow_operations=['GET', 'POST', 'PUT', 'PATCH', 'DELETE']
)


response = requests.get(root+ "/api/tickets")
tickets = response.json()

for t in tickets:
    ticket_id = t["id"]
    ticket_cat = t["category"]
    print("Ticket:", ticket_id)
import json
import websockets

WS_URL = "ws://localhost:3000/ws"

async def post_test_ticket():
    async with websockets.connect(WS_URL) as websocket:
        print("WebSocket connected. Posting test ticket...")

        # Generate ticket JSON using your agent
        ticket_json = agent.invoke("""
Create a brand new support ticket about the product called the "Bean Machine," 
an automated robot arm that prepares custom coffee drinks.

You must invent the situation yourself.

Provide ONLY the following four fields:
- customer_name
- customer_email
- subject
- description

All other ticket fields (id, created_at, category, priority, status, serial_number) 
will be auto-generated by the system and must NOT be included.

The description should clearly describe a problem or user concern related to the Bean Machine’s operation.

Return ONLY a JSON object in this exact format:

{
  "customer_name": "<generated name>",
  "customer_email": "<generated email>",
  "subject": "<short issue summary>",
  "description": "<1-2 sentence explanation of the Bean Machine problem>"
}
""".strip())

        # Send the ticket JSON over the WebSocket
        await websocket.send(ticket_json)
        print("Ticket sent successfully!")

# Run in Jupyter or async context
import asyncio
await post_test_ticket()

------------------- Loaded Key -------------------
Ticket: 1a2b3c4d-0001-0000-0000-000000000001
Ticket: 1a2b3c4d-0002-0000-0000-000000000002
Ticket: 1a2b3c4d-0003-0000-0000-000000000003
Ticket: 1a2b3c4d-0004-0000-0000-000000000004
Ticket: 843628c5-80fe-488b-afd1-12877ad0f29c
Ticket: 4cf79b5d-3512-4da7-bc3a-1eb5040733d6
Ticket: 3a1f8ce7-b0d6-459d-aa44-c7198841a419
Ticket: 0312be73-db7c-4e57-b179-5e301a3f1def
Ticket: 7f41a7d5-33fd-4be6-9d93-729da66e65e7
Ticket: 7842e989-0261-460b-9d39-bf4d9e18da2b
Ticket: 5593ae1e-2014-4a5f-8761-4cfd8eb14671
Ticket: adbe0865-5c00-4afd-a966-53409697df66
Ticket: 22a74a42-8b6f-44b7-a008-3048562f8a90
Ticket: 2cf77c47-5b38-42f5-a0fc-a2d698583055
Ticket: 61f44ec5-b77e-4069-8906-5cd09023d077
Ticket: 3b07931d-183a-4032-9500-fb5d04cdb20d
Ticket: e82f088a-3042-402a-94dc-6ccf41c2814f
Ticket: a19b4806-6e35-4a2a-8008-84a69ef63426
Ticket: ecf0defd-fa56-45b5-8c40-770fe12eb84a
WebSocket connected. Posting test ticket...


> Entering new AgentExecutor chain...
Action: ap

TypeError: data is a dict-like object

## Basic RAG System
The follow cell sets up a basic Retrieval-Augmented Generation (RAG) retriever for the support information. This allows the agent to access relevant support documents when answering user queries, enhancing its ability to provide accurate and helpful responses. It does this by:

1. Loading the support documents from a specified directory.
2. Creating a vector store to index the documents.
3. Demonstrating how to use the retriever to get relevant information based on a user query.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

# Initialize the embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Load documents from a directory
loader = DirectoryLoader("./support-info")
docs = loader.load()

# Initialize the vector store and add documents to it
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(docs)

# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Use the retriever to find the most relevant documents for a given query
query = "Machine won't start."
relevant_docs = retriever.get_relevant_documents(query)

# Print the retrieved documents
for doc in relevant_docs:
    print(doc.page_content)